通常预训练的embedding模型在特定领域往往因缺乏专业知识而表现不佳


微调embedding模型所需要的数据集格式：
```
{
  "query": "什么是市盈率？它如何帮助投资者评估股票价值？",
  "pos": [
    "市盈率（Price-to-Earnings Ratio, P/E Ratio）是衡量股票价格相对于每股收益的指标。计算公式为：市盈率 = 当前股价 / 每股收益(EPS)。它反映了投资者愿意为每一元盈利支付多少价格。",
    "投资者通常使用市盈率来判断股票估值是否合理。较低的市盈率可能意味着股票被低估，而较高的市盈率则可能表示股票被高估或市场预期其未来盈利高速增长。然而，比较市盈率时应考虑行业特性和公司成长阶段。"
  ],
  "neg": [
    "市净率（Price-to-Book Ratio, P/B Ratio）是股价与每股净资产的比率，常用于评估银行、保险等资产密集型公司的价值。",
    "股息收益率（Dividend Yield Ratio）是指公司年度总派息额与当前市价的比率，是衡量股票投资回报的指标之一。",
    "技术分析主要关注股票价格和交易量的历史数据，通过图表模式预测未来价格走势，与基本面分析方法不同。"
  ],
  "prompt": "Represent this sentence for searching relevant documents: ",
  "type": "normal"
}
```


query:str,指的是查询

pos:List[str] 正样本列表

neg:List[str] 负样本列表（与查询不相关，低相关，语义不一致）



正样本的构建：

正样本是指与查询在语义上高度相关或匹配的文本。构建正样本的核心原则是确保与query具有强相关性或语义一致性。这意味着需要从原始数据中准确地抽取或匹配那些能真正回答查询，或与查询语义语义高度一致的内容作为正样本。

正样本的文本粒度：（是选择一个完整的句子、一个段落还是整个文档）需要根据具体的应用场景和模型能力来确定。如果上下文对理解至关重要，那么选择**包含更完整上下文的段落**可能比单个句子更优。


负样本的构建：

负样本是指与查询不相关，低相关或语义不一致的文本，负样本帮助模型区分看上去相似但实际不相关的文本，从而塑造有效的决策边界，防止模型将所有内容都视为相似（即模型坍塌）。模型坍塌是指在训练过程中，模型学习的所有或大部分文本的向量表示都变得非常相似，失去了区分度，导致模型无法有效识别不同文本之间的语义差异。

批内负样本：选择一个批次内其他数据的正样本或query作为负样本，具体选择方法和任务类型有关

难负样本：指那些在语义上或文本表征上与查询具有较高相似度，容易被模型判断为正例，但实际上与查询完全不想管或相关性较低的样本。

挖掘技术：

    - 基于稀疏检索召回：利用如BM25之类的传统词频加权检索算法，为每个查询召回一批高相似度的文档。在移除真正的正例后，剩余的文本可以作为难负样本的候选集。
    - 利用其他模型筛选
    - 结合领域知识或规则。例如，在电商产品推荐领域，同一类但不同品牌的产品，或者功能相似但价格区间、目标用户属性差异显著的产品，都有可能构成有效的难负样本。


难负样本构建示例

```
from rank_bm25 import BM25Okapi
import numpy as np
from sentence_transformers import SentenceTransformer
import jieba  # 添加jieba导入用于中文分词

# 1. 准备语料库
corpus = [
    "市盈率是衡量股票价格相对于每股收益的指标，计算公式为股票价格除以每股收益。",
    "市净率是股价与每股净资产的比率，常用于评估银行等资产密集型公司价值。",
    "股息收益率是公司年度总派息额与股票现价之比，衡量投资回报的指标。",
    "市销率是股票价格与每股销售收入的比值，适用于评估尚未盈利的成长型公司。",
    "企业价值倍数是企业价值与EBITDA的比率，考虑了公司债务水平的估值指标。",
    "现金流折现模型通过预测未来现金流并折现至今来评估公司内在价值。",
    "技术分析主要关注股票价格和交易量的历史数据，预测未来趋势。",
    "基本面分析关注公司财务状况、管理层质量和市场地位等因素。",
    "投资组合理论主张通过资产多样化来分散风险，优化风险回报比。",
    "被动投资策略通过购买指数基金或ETF来追踪特定市场指数表现。"
]

# 查询和已知的正例
query = "什么是市盈率？如何使用它评估股票价值？"
true_positive = corpus[0]  # 第一条关于市盈率的文本是真正的正例

# 2. 使用jieba分词进行BM25检索（稀疏检索阶段）
tokenized_corpus = [list(jieba.cut(doc)) for doc in corpus]  # 对corpus进行分词
tokenized_query = list(jieba.cut(query))  # 对query进行分词

bm25 = BM25Okapi(tokenized_corpus)
bm25_scores = bm25.get_scores(tokenized_query)

# 获取BM25排序后的文档索引（按相关性从高到低排序）
sorted_indices = np.argsort(bm25_scores)[::-1]  # 降序排序
print("BM25检索结果排序:")
for idx in sorted_indices[:5]:  # 取前5名
    print(f"文档{idx} (得分: {bm25_scores[idx]:.4f}): {corpus[idx][:50]}...")

# 3. 使用Embedding模型进行重排序（稠密检索阶段）
# 加载预训练的Embedding模型
model = SentenceTransformer(r'C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3', trust_remote_code=True)  # 示例模型

# 计算查询和所有文档的嵌入向量
query_embedding = model.encode([query])[0]
corpus_embeddings = model.encode(corpus)

# 计算余弦相似度
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity([query_embedding], corpus_embeddings)[0]

# 获取嵌入模型排序后的文档索引
sorted_indices_emb = np.argsort(similarities)[::-1]  # 降序排序
print("\n嵌入模型重排序结果:")
for idx in sorted_indices_emb[:5]:  # 取前5名
    print(f"文档{idx} (相似度: {similarities[idx]:.4f}): {corpus[idx][:50]}...")

# 4. 识别难负样本（高相似度但实际不相关的文档）
# 去除真正的正例
hard_negatives_candidates = [idx for idx in sorted_indices_emb if corpus[idx] != true_positive]

# 从候选中选择前N个作为难负样本
hard_negatives = [corpus[idx] for idx in hard_negatives_candidates[:2]]  # 取前2个作为难负样本

# 5. 最终的训练样本结构
training_sample = {
    "query": query,
    "pos": [true_positive],
    "neg": hard_negatives
}

print("\n最终构建的包含难负样本的训练数据:")
print(f"查询: {training_sample['query']}")
print(f"正例: {training_sample['pos'][0]}")
print("难负样本:")
for i, neg in enumerate(training_sample['neg']):
    print(f"  {i+1}. {neg}")
```

负样本数量通常多于正样本，负样本应覆盖不同类型的不相关情况


对比学习损失函数：InfoNCE

$\mathcal{L} = -\log \frac{\exp(s(x, x^+)/\tau)}{\exp(s(x, x^+)/\tau) + \sum_{x^- \in \mathcal{N}} \exp(s(x, x^-)/\tau)}$



